In [1]:
import rasterio as rio
import geopandas as gpd
import fiona
import os
import pickle
# enable kml file reading for geopandas
from shapely.geometry import box, Polygon
import pandas as pd
import numpy as np
import leafmap.foliumap as leafmap # use folium backend
import rioxarray as rioxr
from typing import List, Union, Dict
from pathlib import Path
from shapely import box
from PIL import Image
import matplotlib.pyplot as plt
import json
fiona.drvsupport.supported_drivers['KML'] = 'rw'


pd.options.display.max_columns = 50

In [2]:
CRS = 32629

In [3]:
project_path = os.path.expanduser("/home/rustt/Documents/Projects/building_damage")

In [4]:
out_dir_path = os.path.join(project_path, "Data/processed")
raw_data_path = os.path.join(project_path, "Data/raw")
town_loc_path = os.path.join(raw_data_path, "villages_location/Earthquake_marocco_town.kml")

In [5]:
def make_path(file_name, dir=out_dir_path):
    return os.path.join(dir, file_name)

def save_pickle(data, output_path):
    with open(output_path, 'wb') as f:
        pickle.dump(data, f)
        
def load_pickle(path):
    with open(path, 'rb') as f:
        data = pickle.load(f)
    return data

In [6]:
with open(os.path.join(raw_data_path, "xview_geotransforms.json")) as fp:
    xview = json.load(fp)

In [7]:
type(xview)

dict

In [8]:
len(xview)

22068

In [9]:
list(xview.keys())[:2]

['guatemala-volcano_00000020_pre_disaster.png',
 'guatemala-volcano_00000020_post_disaster.png']

### xview[]

In [16]:
xview["guatemala-volcano_00000020_pre_disaster.png"]

[[-90.82307020932015,
  4.488628277641187e-06,
  0.0,
  14.414359902299086,
  0.0,
  -4.488628277641187e-06],
 'GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433],AUTHORITY["EPSG","4326"]]']

In [17]:
items = [_ for _ in xview.items()]

In [18]:
items[:5]

[('guatemala-volcano_00000020_pre_disaster.png',
  [[-90.82307020932015,
    4.488628277641187e-06,
    0.0,
    14.414359902299086,
    0.0,
    -4.488628277641187e-06],
   'GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433],AUTHORITY["EPSG","4326"]]']),
 ('guatemala-volcano_00000020_post_disaster.png',
  [[-90.82307020932015,
    4.488628277641187e-06,
    0.0,
    14.414359902299086,
    0.0,
    -4.488628277641187e-06],
   'GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433],AUTHORITY["EPSG","4326"]]']),
 ('guatemala-volcano_00000022_pre_disaster.png',
  [[-90.84656734507239,
    4.488484900944135e-06,
    0.0,
    14.391883642640861,
    0.0,
    -4.488484900944135e-06],
   'GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6